# Econ 8210 Quant Macro, Homework 1
## Part 1 - Numerical Integration and Optimization
Haosi Shen, Fall 2024

In [1]:
# Housekeeping
import numpy as np
import pandas as pd
import time

np.random.seed(42) 

## 1. Integration

Compute
\begin{equation*}
\int_{0}^{T} e^{-\rho t} u(1-e^{-\lambda t})\,dt
\end{equation*}
for $T=100$, $\rho = 0.04$, $\lambda = 0.02$, and $u(c)=-e^{-c}$ using **quadrature** (midpoint, Trapezoid, and Simpson rule) and Monte Carlo integration.

In [2]:
# Define Problem
T = 100
rho = 0.04
lambda_ = 0.02

def u(c):
    return -np.exp(-c)

def integrand(t):
    return np.exp(-rho * t) * u(1 - np.exp(-lambda_ * t))

# Number of intervals/draws
n_intervals = np.array([10, 100, 1000, 10000, 100000])

### Quadrature Integration

In [3]:
# Midpoint
def midpoint_quadrature(a, b, n):
    start_time = time.time() 

    h = (b - a) / n
    total = 0
    for i in range(n):
        midpoint = a + (i + 0.5) * h
        total += integrand(midpoint)

    integral_est = h * total
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_midpoint =  np.vectorize(midpoint_quadrature)
results_midpoint, times_midpoint = vec_midpoint(0, T, n_intervals)

In [4]:
# Trapezoid
def trapezoid_quadrature(a, b, n):
    start_time = time.time() 
    
    h = (b - a) / n
    total = 0.5 * (integrand(a) + integrand(b))
    for i in range(1, n):
        total += integrand(a + i * h)
    
    integral_est = h * total
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_trapezoid =  np.vectorize(trapezoid_quadrature)
results_trapezoid, times_trapezoid = vec_trapezoid(0, T, n_intervals)

In [5]:
# Simpson's Rule
def simpsons_quadrature(a, b, n):
    start_time = time.time() 
    
    if n % 2 == 1:
        n += 1  # ensure n is even
    h = (b - a) / n
    total = integrand(a) + integrand(b)
    for i in range(1, n, 2):
        total += 4 * integrand(a + i * h)
    for i in range(2, n, 2):
        total += 2 * integrand(a + i * h)
    
    integral_est = (h / 3) * total
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_simpsons =  np.vectorize(simpsons_quadrature)
results_simpsons, times_simpsons = vec_simpsons(0, T, n_intervals)

### Monte Carlo Integration

In [6]:
def monteCarlo_integration(a, b, n):
    start_time = time.time() 
    
    random_points = np.random.uniform(a, b, n)
    integral_est = (b - a) * np.mean([integrand(t) 
                                           for t in random_points])
    end_time = time.time()
    comp_time = end_time - start_time # record computation time
    return integral_est, comp_time

vec_monteCarlo =  np.vectorize(monteCarlo_integration)
results_monteCarlo, times_monteCarlo = vec_monteCarlo(0, T, n_intervals)

### Results

In [7]:
results_integration = pd.DataFrame(np.stack((results_midpoint, results_trapezoid, 
                                             results_simpsons, results_monteCarlo)),
            columns = ['N = 10', 'N = 100', 'N = 1000', 'N = 5000', 'N = 10000'], 
            index = (['Midpoint', 'Trapezoid', 'Simpson\'s', 'Monte Carlo']))


print("Integral Estimates")
display(results_integration)

Integral Estimates


,N = 10,N = 100,N = 1000,N = 5000,N = 10000
Midpoint,-17.964420,-18.207039,-18.209501,-18.209525,-18.209525
Trapezoid,-18.702748,-18.214498,-18.209575,-18.209526,-18.209525
Simpson's,-18.224641,-18.209527,-18.209525,-18.209525,-18.209525
Monte Carlo,-24.732456,-20.260672,-18.809211,-18.360223,-18.198811


In [8]:
times_integration = pd.DataFrame(np.stack((times_midpoint, times_trapezoid, 
                                             times_simpsons, times_monteCarlo)),
            columns = ['N = 10', 'N = 100', 'N = 1000', 'N = 5000', 'N = 10000'], 
            index = (['Midpoint', 'Trapezoid', 'Simpson\'s', 'Monte Carlo']))

print("Computation Time (seconds)")
display(times_integration)

Computation Time (seconds)


,N = 10,N = 100,N = 1000,N = 5000,N = 10000
Midpoint,0.000047,0.000222,0.002029,0.015107,0.140622
Trapezoid,0.000018,0.000140,0.001393,0.014129,0.139786
Simpson's,0.000020,0.000148,0.001448,0.014500,0.143689
Monte Carlo,0.000036,0.000157,0.001440,0.014622,0.174707


In alignment with the theoretical properties of each method,
> * The quadrature methods provide accurate results as the number of intervals $N$ increases, with Simpson's rule converging the fastest.
> * Monte Carlo integration has more variability but still trends toward the true value with higher number of draws $N$.
> * Regarding computation time, Midpoint and Simpson’s methods are generally faster and more efficient. Monte Carlo integration becomes competitive at larger $N$, while the trapezoid rule is generally slower.

In general, quadrature methods are faster and more accurate for lower-dimension problems and smaller $N$, whereas Monte Carlo becomes more competitive at large $N$ and higher dimensions. 

## 2. Optimization: Rosenbrock function

\begin{equation}
\min_{x, y}\; 100(y-x^2)^2 + (1-x)^2
\end{equation}

Using the Newton-Raphson, Broyden-Fletcher-Goldfarb-Shanno (BFGS), steepest (gradient) descent, and conjugate descent methods.

> The global minimum is at $(x,y)=(1,1)$, where $f(x,y)=0$.

In [9]:
# Define the function and its gradient
def rosenbrock(x, y):
    return 100 * (y - x**2)**2 + (1 - x)**2


def gradient_rosenbrock(x, y):
    df_dx = -400 * x * (y - x**2) - 2 * (1 - x)
    df_dy = 200 * (y - x**2)
    return np.array([df_dx, df_dy])

### Vanilla Gradient Descent (i.e. Steepest Descent)

In [25]:
def steepest_descent(init_x, init_y, alpha = 0.001, tol = 1e-6, max_iter = 10000):
    # initial guess
    x, y = init_x, init_y

    for i in range(max_iter):
        grad = gradient_rosenbrock(x, y)
        norm_grad = np.linalg.norm(grad)
        
        # Check for convergence
        if norm_grad < tol:
            break

        # Normalized direction of steepest descent
        d = -grad / norm_grad

        # Reduce step size with decay
        curr_alpha = alpha / (1 + 0.1 * i)

        # Update x and y
        x -= curr_alpha * d[0]
        y -= curr_alpha * d[1]
    
    return x, y, rosenbrock(x, y), i

In [31]:
init_x, init_y = 0.0, 0.0  # initial guess
alpha = 0.00001    # Step size, fixed
tol = 1e-6    # convergence tolerance

x_min, y_min, f_min, num_iters = steepest_descent(init_x, init_y, alpha, tol)

print(f"Minimum point: x = {x_min}, y = {y_min}")
print(f"Function value at minimum: f(x, y) = {f_min}")
print(f"Number of iterations: {num_iters}")

Minimum point: x = -0.0006959537330666208, y = -1.141741965577476e-08
Function value at minimum: f(x, y) = 1.0013923918423107
Number of iterations: 9999


line search

In [33]:
from scipy.optimize import minimize_scalar

def steepest_descent_line_search(initial_x, initial_y, tol=1e-6, max_iter=10000):
    x, y = initial_x, initial_y

    for i in range(max_iter):
        grad = gradient_rosenbrock(x, y)
        norm_grad = np.linalg.norm(grad)

        # Check convergence
        if norm_grad < tol:
            break

        # Calculate normalized descent direction
        d = -grad / norm_grad

        # Define a function for line search along direction d
        def f_alpha(alpha):
            x_new = x + alpha * d[0]
            y_new = y + alpha * d[1]
            return rosenbrock(x_new, y_new)

        # Perform line search to find optimal alpha
        res = minimize_scalar(f_alpha, bounds=(0, 1), method='bounded')

        # Update step size with line search result
        alpha = res.x

        # Update x and y
        x += alpha * d[0]
        y += alpha * d[1]

    return x, y, rosenbrock(x, y), i

# Initial guess and parameters
initial_x, initial_y = 0.0, 0.0  # Closer initial guess
tol = 1e-6  # Convergence tolerance

# Solve using steepest descent with line search
x_min, y_min, f_min, num_iters = steepest_descent_line_search(initial_x, initial_y, tol=tol)

# Display results
print(f"Minimum point: x = {x_min}, y = {y_min}")
print(f"Function value at minimum: f(x, y) = {f_min}")
print(f"Number of iterations: {num_iters}")


Minimum point: x = 1.0000006569245152, y = 0.9999996655439365
Function value at minimum: f(x, y) = 2.721226603994853e-10
Number of iterations: 9999
